# Disaster Tweets Classification - Mini Project

## Problem and Data Overview

Natural Language Processing (NLP) involves developing algorithms that allow computers to understand, interpret, and generate human language.
This competition challenges us to classify tweets as being related to a real disaster (1) or not (0).
The train.csv contains labeled examples (text + target) while test.csv contains only texts.
Tweets are short and informal, containing many misspellings, slang, and abbreviations, making this a realistic NLP task.
Understanding disaster-related tweets can aid in faster emergency responses and resource allocation.

## Load Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import string

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Load Data

In [ ]:
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

print("Training data shape:", train_df.shape)
print("Sample training data:")
print(train_df.head())

## Exploratory Data Analysis (EDA)
### In EDA, we plan to:
- Analyze the distribution of the target variable to check for class imbalance.
- Check for missing values across features.
- Explore the text length to determine appropriate sequence padding.
- Visualize common words in disaster vs non-disaster tweets to understand text patterns.

Based on the observations from EDA, we will determine data cleaning and modeling strategies.

In [ ]:
# Observing target distribution to check for class imbalance
sns.countplot(data=train_df, x='target')
plt.title('Distribution of Target Variable (0 = Not Disaster, 1 = Disaster)')
plt.xlabel('Target')
plt.ylabel('Count')
plt.show()

### Observation:
- There is a slight imbalance (~57% non-disaster, ~43% disaster), but it is manageable.
- Slight imbalance may still slightly bias the model towards non-disasters.

In [ ]:
# Checking for missing values
print("Missing values in training data:")
print(train_df.isnull().sum())

### Observation:
- Location (approx 33% missing) and keyword (approx 20% missing) columns have substantial missingness.
- Since text has no missing values and is critical for this project, we can proceed safely.

In [ ]:
# Analyzing text lengths
train_df['text_len'] = train_df['text'].apply(lambda x: len(x.split()))
sns.histplot(train_df['text_len'], bins=30)
plt.title('Distribution of Tweet Lengths')
plt.xlabel('Number of Words')
plt.show()

### Observation:
Most tweets are under 30 words, reaffirming the decision to limit max sequence length.

In [ ]:
# Generate Word Clouds to visualize common words
from wordcloud import WordCloud

def plot_wordcloud(text, title):
    wc = WordCloud(width=800, height=400, background_color='white').generate(' '.join(text))
    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)
    plt.show()

plot_wordcloud(train_df[train_df['target']==1]['text'], 'Disaster Tweets')
plot_wordcloud(train_df[train_df['target']==0]['text'], 'Non-Disaster Tweets')

### Observation:
- Disaster tweets highlight words like "fire", "earthquake", "rescue", "evacuate".
- Non-disaster tweets often use disaster-related terms humorously or figuratively.

### Plan of Analysis based on EDA observations:
- Since text column is complete and crucial, we'll focus our modeling on the cleaned text.
- Given most tweets are short, we'll pad sequences to a maximum length of 100 tokens.
- Missing keywords and location data will not be used in modeling to avoid introducing noise.
- GloVe embeddings will be used to leverage semantic relationships between words.
- Bidirectional LSTM will be selected to capture context from both directions in sequential data.

## Data Cleaning and Preprocessing

Cleaning text by removing noise like URLs, HTML tags, special characters, and lowercasing text.


Decided not to remove stopwords because LSTM models benefit from preserving the sequential grammatical structure.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(r'<.*?>', '', text)  # remove HTML
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # keep letters only
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    text = text.strip()
    return text

train_df['text_clean'] = train_df['text'].apply(clean_text)
test_df['text_clean'] = test_df['text'].apply(clean_text)

### Observation:
Text cleaning reduces noise and ensures the model focuses on meaningful features.

## Word Embedding Strategy

-Word embeddings convert text into a numerical format usable by neural networks.
- TF-IDF captures word importance but ignores word order.
- GloVe (Global Vectors) embeddings capture semantic relationships between words by using statistical information about co-occurrence.
- This will allow the LSTM to better understand the meaning behind sequences of words.

In [ ]:
# Tokenizing the text
vocab_size = 20000
max_length = 100

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['text_clean'])

X = tokenizer.texts_to_sequences(train_df['text_clean'])
X = pad_sequences(X, maxlen=max_length)

X_test = tokenizer.texts_to_sequences(test_df['text_clean'])
X_test = pad_sequences(X_test, maxlen=max_length)

y = train_df['target'].values

In [ ]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Load GloVe embeddings
embedding_index = {}

with open('/kaggle/input/glove-6b-50d/glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

embedding_dim = 50
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


### Observation:
GloVe embeddings enable our model to have rich prior knowledge of word relationships, improving generalization.

## Model Architecture

- Bidirectional LSTM is chosen because it processes the sequence both forward and backward.
- This is critical for tweets, where important context might appear at the start or end of the text.
- Dropout layers are added to prevent overfitting by randomly turning off neurons during training.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

### Observation:
Model architecture is designed to balance complexity and regularization for robust performance.

## Model Training

In [ ]:
# EarlyStopping is used to halt training once validation performance stops improving.
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=10, batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])

## Results and Evaluation

In [ ]:
# Plotting accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plotting loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluation on Validation Set
val_preds = (model.predict(X_val) > 0.5).astype('int32')
print("Validation Set Classification Report:")
print(classification_report(y_val, val_preds))

### Observations:
- Training and validation accuracy follow each other closely, suggesting low overfitting.
- Validation F1-score is balanced across both classes.
- Slight performance drop in validation is normal, indicating good generalization.

## Kaggle Submission

In [ ]:
test_preds = (model.predict(X_test) > 0.5).astype('int32')

submission = pd.DataFrame({'id': test_df['id'], 'target': test_preds.flatten()})
submission.to_csv('/kaggle/working/submission.csv', index=False)

# Submission file created and ready to upload to Kaggle.

## Conclusion

### Key Learnings:
- Thorough EDA guided correct preprocessing and modeling decisions.
- Cleaning and GloVe embeddings significantly enhanced model performance.
- Bidirectional LSTM captured tweet sequence information effectively.
- Regularization techniques like Dropout and EarlyStopping helped prevent overfitting.

### Limitations:
- Tweets are noisy and sometimes sarcastic or figurative, making them harder to classify.

### Future Improvements:
- Fine-tune embeddings during training.
- Try using GloVe 100d/300d vectors for more detail.
- Implement Transformer-based models (e.g., BERT) for better sequence modeling.

## 10. References

- Kaggle Disaster Tweets Competition: https://www.kaggle.com/competitions/nlp-getting-started
- Stanford GloVe Project: https://nlp.stanford.edu/projects/glove/
- Tensorflow/Keras API: https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM
- Scikit-learn Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html